# 数据聚合与分组运算

In [1]:
import numpy as np
import pandas as pd



------GroupBy机制-----

In [14]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],'key2' : ['one', 'two', 'one', 'two', 'one'], 
                   'data1' : np.random.randn(5),'data2' : np.random.randn(5)})
print(df)
df.groupby(['key1','key2']).mean()

  key1 key2     data1     data2
0    a  one -0.508397  0.545625
1    a  two  1.000162 -0.876884
2    b  one -0.689579 -0.664219
3    b  two  0.889430  0.159773
4    a  one  0.314674 -0.674533


data1     data2
key1 key2                    
a    one  -0.096862 -0.064454
     two   1.000162 -0.876884
b    one  -0.689579 -0.664219
     two   0.889430  0.159773

In [13]:
states = np.array(['Ohio', 'California', 'California','Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005   -1.011853
            2006    0.356701
Ohio        2005   -0.360712
            2006   -0.059333
Name: data1, dtype: float64

In [18]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.508397  0.545625
1    a  two  1.000162 -0.876884
4    a  one  0.314674 -0.674533
b
  key1 key2     data1     data2
2    b  one -0.689579 -0.664219
3    b  two  0.889430  0.159773


In [27]:
piece=dict(list(df.groupby(['key1', 'key2'])))
piece['a','one']

,key1,key2,data1,data2
0,a,one,-0.508397,0.545625
4,a,one,0.314674,-0.674533


In [29]:
df.groupby(df.dtypes, axis=1)

In [60]:
df.groupby(['key1', 'key2'])[['data1']].mean()

data1
key1 key2          
a    one  -0.096862
     two   1.000162
b    one  -0.689579
     two   0.889430

In [84]:
# 通过字典或Series进行分组
people = pd.DataFrame(np.random.randn(5, 5),columns=['a', 'b', 'c', 'd', 'e'],index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1,2]]=np.nan
print(people)
mapping = {'a': 'red', 'b': 'red', 'c': 'blue','d': 'blue', 'e': 'red', 'f' : 'orange'}
by_column=people.groupby(mapping,axis=1)
by_column.mean()

               a         b         c         d         e
Joe    -0.281985  0.295335 -0.085396  1.423141 -1.306623
Steve   1.975341 -0.828576 -0.461201 -2.867827  0.798054
Wes    -0.082412       NaN       NaN  1.348343 -0.334425
Jim     0.311122  1.687523 -1.274849 -0.131851 -0.903621
Travis -0.260772 -1.213456 -0.631235  2.294268 -0.488275


,blue,red
Joe,0.668873,-0.431091
Steve,-1.664514,0.648273
Wes,1.348343,-0.208419
Jim,-0.703350,0.365008
Travis,0.831517,-0.654168


In [85]:
# 通过函数进行分组
people.groupby(len).sum()

,a,b,c,d,e
3,-0.053275,1.982858,-1.360245,2.639634,-2.544669
5,1.975341,-0.828576,-0.461201,-2.867827,0.798054
6,-0.260772,-1.213456,-0.631235,2.294268,-0.488275


In [86]:
# 将函数跟数组、列表、字典、Series混合使用也不是问题，因为任何东西在内部都会被转换为数组
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()


a         b         c         d         e
3 one -0.281985  0.295335 -0.085396  1.348343 -1.306623
  two  0.311122  1.687523 -1.274849 -0.131851 -0.903621
5 one  1.975341 -0.828576 -0.461201 -2.867827  0.798054
6 two -0.260772 -1.213456 -0.631235  2.294268 -0.488275

In [101]:
# 根据索引级别分组
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US','JP', 'JP'],[1, 3, 5, 1, 3]],names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns, index=np.arange(1,5))
print(hier_df)
hier_df.groupby(level='cty', axis=1).count()

cty          US                            JP          
tenor         1         3         5         1         3
1     -0.283363  0.890189  1.947411 -0.665872 -0.002976
2      1.100341 -2.259698  1.685473 -0.763302  1.636287
3     -1.292423 -0.101708 -0.195962 -0.235965 -1.605443
4      0.293681 -1.000305 -0.719099  0.256692  0.136228


cty,JP,US
1,2,3
2,2,3
3,2,3
4,2,3


-----数据聚合-----

In [159]:
# 面向列的多函数应用，对不同的列使用不同的聚合函数，或一次应用多个函数
tips=pd.read_csv('E:\\useFiles\\examples\\tips.csv')
tips['tip_pct'] = tips['tip'] / tips['total_bill']
grouped=tips.groupby(['day','smoker'])
grouped['tip_pct'].agg([('foo', 'mean'), ('bar', 'std')])     #describe

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [147]:
# 对于DataFrame，定义一组应用于全部列的一组函数
functions=[('js','count'),'mean','max']
grouped['tip_pct','total_bill'].agg(functions)

tip_pct                     total_bill                  
                 js      mean       max         js       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [154]:
# 假设你想要对一个列或不同的列应用不同的函数。具体的办法是向agg传入一个从列名映射到函数的字典
# grouped.agg({'tip':'max', 'size':'sum'})
grouped.agg({'tip_pct':['min', 'max', 'mean', 'std'], 'size':'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

In [150]:
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [155]:
# 以“没有行索引”的形式返回聚合数据
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


-----apply：一般性的“拆分—应用—合并”-----

In [188]:
# 根据分组选出最高的n个值
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]
top(tips,7)
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [189]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [194]:
# 分位数和桶分析
frame = pd.DataFrame({'data1': np.random.randn(1000),'data2': np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4)
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),'count': group.count(), 'mean': group.mean()}
grouped=frame.data2.groupby(quartiles)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-2.966, -1.336]",93.0,1.977675,-0.056454,-3.448788
"(-1.336, 0.287]",515.0,2.731071,0.008442,-3.613970
"(0.287, 1.91]",369.0,3.274503,0.034981,-2.422503
"(1.91, 3.533]",23.0,2.710891,0.316404,-2.263897


In [197]:
# 要根据样本分位数得到大小相等的桶，使用qcut
grouping = pd.qcut(frame.data1, 4)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-2.96, -0.707]",250.0,2.731071,0.049489,-3.448788
"(-0.707, 0.0145]",250.0,2.494036,-0.039992,-3.613970
"(0.0145, 0.669]",250.0,3.274503,-0.032110,-2.367204
"(0.669, 3.533]",250.0,2.771551,0.099744,-2.422503


In [205]:
states = ['Ohio', 'New York', 'Vermont', 'Florida','Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index=states)
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
print(data)
print(data.groupby(group_key).mean())
# 用分组平均值去填充NA值
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio         -2.220588
New York     -0.026868
Vermont            NaN
Florida      -0.276915
Oregon        0.582641
Nevada             NaN
California    0.636822
Idaho              NaN
dtype: float64
East   -0.841457
West    0.609731
dtype: float64


Ohio         -2.220588
New York     -0.026868
Vermont      -0.841457
Florida      -0.276915
Oregon        0.582641
Nevada        0.609731
California    0.636822
Idaho         0.609731
dtype: float64

In [206]:
# 在代码中预定义各组的填充值，使用name属性
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio         -2.220588
New York     -0.026868
Vermont       0.500000
Florida      -0.276915
Oregon        0.582641
Nevada       -1.000000
California    0.636822
Idaho        -1.000000
dtype: float64

In [249]:
# 随机采样和排列
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)
deck = pd.Series(card_val, index=cards)
print(deck[:14])
# 从整副牌中抽出n张
def draw(deck, n=5):
    return deck.sample(n)
draw(deck, 3)
# 从每种花色中随机抽取n张
get_suit = lambda card: card[-1]
deck.groupby(get_suit).apply(draw, n=1)

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
AS      1
dtype: int64


C  QC    10
D  9D     9
H  KH    10
S  JS    10
dtype: int64

In [287]:
# 分组加权平均数
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a','b', 'b', 'b', 'b'],'data': np.random.randn(8),'weights': np.random.rand(8)})
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
get_mean = lambda g: np.mean(g['data'])
print(grouped.apply(get_wavg))
print(grouped.apply(get_mean))

category
a   -1.003054
b    1.268110
dtype: float64
category
a   -0.788545
b    1.136158
dtype: float64


array([10, 14, 12, 11, 12, 19, 19, 14, 11, 16, 12, 16, 18, 15, 10, 13, 16,
       18, 11, 16])

In [338]:
# 计算一个由日收益率（通过百分数变化计算）与SPX之间的年度相关系数组成的DataFrame
close_px = pd.read_csv('E:\\useFiles\\examples\\stock_px_2.csv', parse_dates=True,index_col=0)
print(close_px[-3:])
spx_corr = lambda x: x.corrwith(x['SPX'])
rets = close_px.pct_change().dropna()
get_year = lambda x: x.year
by_year = rets.groupby(get_year)

by_year.apply(spx_corr)
by_year.apply(lambda g:g['AAPL'].corr(g['MSFT']))

              AAPL   MSFT    XOM      SPX
2011-10-12  402.19  26.96  77.16  1207.25
2011-10-13  408.43  27.18  76.37  1203.66
2011-10-14  422.00  27.27  78.11  1224.58


2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

In [362]:
#组级别的线性回归
# 定义regress函数（利用statsmodels计量经济学库）对各数据块执行普通最小二乘法回归
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params
# 按年计算AAPL对SPX收益率的线性回归:
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


-----透视表pivot_table-----

In [379]:
print(tips[:3])
tips.pivot_table(index=['day', 'smoker'])

# 只聚合tip_pct和size，而且想根据time和day进行分组，将smoker放到列上
# 传入margins=True添加分项小计，将会添加标签为All的行和列，其值对应于单个等级中所有数据的分组统计
tips.pivot_table(['tip_pct', 'size'],  index=['time', 'day'], columns='smoker', margins=True)

   total_bill   tip smoker  day    time  size   tip_pct
0       16.99  1.01     No  Sun  Dinner     2  0.059447
1       10.34  1.66     No  Sun  Dinner     3  0.160542
2       21.01  3.50     No  Sun  Dinner     3  0.166587


size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [400]:
# 要使用其他的聚合函数，将其传给aggfunc，例如使用count或len可以得到有关分组大小的交叉表（计数或频率）
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day', aggfunc=len, margins=True )
tips.pivot_table('tip_pct', index=['time','size', 'smoker'], columns='day', aggfunc='mean', fill_value=0 )

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

In [401]:
# 交叉表crosstab
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244